# Kats 205 Forecasting with Global Model

This tutorial will introduce how to use the global model in Kats.  The global model is a new and powerful forecasting method that combines exponential smoothing models with recurrent neural networks, resulting in higher accuracy than other approaches. The table of contents for Kats 205 is as follows:

1. Overview of Global Model for Forecasting  
2. Build Your Own Global Model or Global Ensemble From Scratch  
    2.1 Introduction to `GMParam`  
    2.2 Forecasting using a single global model with `GMModel`  
    2.3 Forecasting using a global model ensemble with `GMEnsemble`  
    2.4 Backtesting with `GMBacktester`  
3. Using Pretrained Global Model or Global Ensemble  

**Note:** We provide two types of tutorial notebooks
- **Kats 101**, basic data structure and functionalities in Kats 
- **Kats 20x**, advanced topics, including advanced forecasting techniques, advanced detection algorithms, `TsFeatures`, meta-learning, global model etc. 

## 1. Overview of Global Model for Forecasting

The Global Model, henceforth abbreviated as GM, is a powerful forecasting model that was originally proposed by Slawek Smyl and won the Computational Intelligence in Forecasting International Time Series Competition (2016) and the M4 Forecasting competition (2018).  The GM effectively combines exponential smoothing models with neural networks in a way that results in higher accuracy than a method that only uses pure statistics or machine learning.  

Kats has many forecasting models that only use pure statistics, and we discussed many of these approaches in [Kats 201](kats_201_forecasting.ipynb).  We also provide an ML approach to forecasting in our metalearning framework for forecasting, which is covered in [Kats 204](kats_204_metalearning.ipynb).  The GM approach generates more accurate forecasts by combining the advantages of pure statistical and ML approaches.  For more details on how this works, please see the [original paper for the GM](https://www.sciencedirect.com/science/article/pii/S0169207019301153).

GM is trained with a large number of time series of the same time granularity.  It generally supports batch processing, meaning it can efficiently generate forecasts for several time series at the same time.  


In Kats, we build upon the original model and allow support for two different neural network types when building a GM:
1. **Recurrent neural network (RNN)**: This is the default and it's best for short-term forecasting.  
2. **Sequence to Sequence (S2S)**: More optimal for medium to long term forecasting. 

## 2. Build Your Own Global Model or Global Ensemble From Scratch

The `GMModel` is our basic class to build a single GM.  Kats also supports Global Model Ensembles (GMEs), which are ensembles of independent GMs, with the `GMEnsemble` class.  The `GMParam` is the parameter class for both `GMModel` and `GMEnsemble`.  We also provide the `GMBacktester` class for parameter tunning and backtesting. 

The examples in this section are designed to display the basic functionality of each of the aforementioned classes.  They are of limited scale and not expected to provide good performance.

### 2.1 Introduction to `GMParam`

All parameters for a GM or a GME are specified using the `GMParam` class.  The `GMParam` class does basic parameter checking when initialized to ensure that the parameters are correctly specified.  Here are some of its key arguments:

* **freq**: `str` or `pd.Timedelta`, The time granularity of the model (and the input time series.) For example, `freq='D'` indicates a daily model;
* **model_type**: `str`, The name of neural network type - either 'rnn' (recurrent neural network) or 's2s' (sequence to sequence). Default is 'rnn';
* **seasonality**: `int`, The integer length of the seasonality period. The default value is 1, indicating a non-seasonal model;
* **input_window**: `int`, This parameter specifies the size into which we segment our input time series to feed them into the neural network.  It should be greater than the `seasonality` argument;
* **fcst_window**: `int`,  The number of data points forecast in a single forecast step;
* **quantile**: `list[float]`, The float values of the quantiles to forecast.  The first value of this list should always be 0.5, representing the median.  The default value is `[0.5,0.05,0.95,0.99]`;
* **nn_structure**: `list[list[int]]`, The structure of the neural network. If not specified, the default value is `[[1,3]]`;
* **loss_function**: `list[str]`, The name of loss function - either 'pinball' or 'adjustedpinball';
* **gmfeature**: `list[str]` or `str`; A single or a list of feature names.

For the definition of other parameters, please see our documentation.

In [1]:
import numpy as np
import pandas as pd
import sys
import warnings
import os
import pprint

warnings.simplefilter(action='ignore')
sys.path.append("../")

from kats.models.globalmodel.utils import GMParam

Below we initialize a `GMParam` instance that we will use to train a daily model with weekly seasonality. 

In [2]:
gmparam = GMParam(
    input_window = 35, 
    fcst_window = 31,
    seasonality = 7,
    freq = 'D',
    loss_function = 'adjustedpinball',
    nn_structure = [[1,3]],
    gmfeature = ['last_date'],
    epoch_num = 1, 
    epoch_size = 2, # use a small num just for demonstration
    gmname = "daily_default",
)

### 2.2 Forecasting using a single global model with `GMModel`  

In [3]:
from kats.models.globalmodel.model import GMModel, load_gmmodel_from_file
from kats.models.globalmodel.serialize import global_model_to_json, load_global_model_from_json
from kats.consts import TimeSeriesData

Now we are ready to train a global model.  A `GMModel` object can be initialized with just one parameter - an instance of the `GMParam` object.

In [4]:
gm = GMModel(gmparam)

To train a `GMModel` object, we need a list or a dictionary of `TimeSeriesData` objects. We will simulate two dictionaries, one for training and one for testing, using the `get_ts` method from our test functions.

In [5]:
from kats.tests.test_globalmodel import get_ts

train_TSs = [get_ts(n*5, '2020-05-06', has_nans=False) for n in range(20, 40)]
test_TSs = [get_ts(n*2, '2020-05-06', has_nans=False) for n in range(40, 45)]

It is straightforward to train the GM using the `train` function.  This function also saves basic information about the training process, which we look at below.

In [6]:
# train the model
training_info = gm.train(train_TSs)

#training_info saves the information of training process
pprint.pprint(training_info)

{'train_loss_monitor': [0.10751946],
 'train_loss_val': [0.2553587183356285],
 'valid_fcst_monitor': [],
 'valid_loss_monitor': [{'epoch': 0}]}


Now we can use the trained model to generate forecasts using the `predict` function.  The input of the `predict` function can either be a single `TimeSeriesData` or a list/dictionary of them.  The `predict` function also requires you to specify the number of steps you wish to forecast.  Here, we demonstrate doing batch forecasting on the 5 `TimeSeriesData` objects in `test_TSs` for 3 steps.


In [7]:
fcsts = gm.predict(test_TSs, steps = 3)

This generates a dictionary with 5 keys, one for each time series in `test_TSs`.

In [8]:
fcsts.keys()

dict_keys([0, 1, 2, 3, 4])

The values in the dictionary are the 3-step forecasts for each time series in `test_TSs`.  Let's look at the forecast for the 3rd time series in the list.  We see a `pd.DataFrame` that gives forecasts for each percentiles 5, 50, 95, and 99 in the 3 days after the time series ends.

In [9]:
fcsts[3]

,fcst_quantile_0.5,fcst_quantile_0.05,fcst_quantile_0.95,fcst_quantile_0.99,time
0,-0.748453,-0.189885,-0.567380,-0.067029,2020-07-31
1,0.918473,0.573938,0.905266,1.170326,2020-08-01
2,0.627886,0.869479,0.085866,0.972686,2020-08-02


We can save the model using the `save_model` function.

In [10]:
# save model
gm.save_model("gm_example_1.p")

Let's load the saved model and again use it to repeat the forecast we did above.

In [11]:
# load model
gm2 = load_gmmodel_from_file("gm_example_1.p")

# make prediction 
fcsts2 = gm2.predict(test_TSs, steps = 3)
fcsts2[3]

,fcst_quantile_0.5,fcst_quantile_0.05,fcst_quantile_0.95,fcst_quantile_0.99,time
0,-0.748453,-0.189885,-0.567380,-0.067029,2020-07-31
1,0.918473,0.573938,0.905266,1.170326,2020-08-01
2,0.627886,0.869479,0.085866,0.972686,2020-08-02


Now let's remove the saved model.

In [12]:
os.remove("gm_example_1.p")

We can also encode GM into a json string using the  `global_model_to_json` function.

In [13]:
gm_str = global_model_to_json(gm)

Let's repeat the same prediction one more time.

In [14]:
# load model from json string
gm3 = load_global_model_from_json(gm_str)

# make prediction 
fcsts3 = gm3.predict(test_TSs, steps = 3)
fcsts3[3]

,fcst_quantile_0.5,fcst_quantile_0.05,fcst_quantile_0.95,fcst_quantile_0.99,time
0,-0.748453,-0.189885,-0.567380,-0.067029,2020-07-31
1,0.918473,0.573938,0.905266,1.170326,2020-08-01
2,0.627886,0.869479,0.085866,0.972686,2020-08-02


### 2.3 Forecasting using a global model ensemble with `GMEnsemble`

You can also easily build an ensemble of several independently trained GMs with `GMEnsemble` class.  We initialize the `GMEnsemble` class with the same `GMParam` object that we use to initialize a single GM, but there are a few other parameters needed to indicate how to create the ensemble.

Here is the list of attributs needed to initialize GMEnsemble`:
* **gmparam**: `GMParam`; the parameters for each GM in the ensemble;
* **ensemble_type**: `str`, how to aggregate the forecasts - either 'median' or 'mean'.  Default is 'median';
* **splits**: `int`, the number of sub-datasets into which the training data is to be partitioned.  Default is 3;
* **overlap**: `bool`, Whether or not sub-datasets overlap with each other or not. If True, each training examples appears in `splits-1` sub-datasets.  Otherwise, each training example appears in only 1 sub-dataset.  Default is True;
* **replicate**: `int`, The number of GMs in the ensemble to be trained on each sub-dataset. Default is 1;
* **multi**: `bool`, whether or not to use multi-processing for training and prediction. Default is False.

Note that a `GMEnsemble` object will build `splits*replicate` independent `GMModel` objects, and the final forecasts are aggregated from the forecasts generated from each trained `GMModel` object.

In [15]:
from kats.models.globalmodel.ensemble import GMEnsemble, load_gmensemble_from_file

We can initialize a `GMEnsemble` object as follows.

In [16]:
gme = GMEnsemble(gmparam, splits=3, overlap=True, replicate=1, multi=True)

Now we can train the `GMEnsemble` object. Note that one has the choice of setting aside a test set from the training data to measure the performance of each `GMModel` object throughout the training process; we do so using the `test_size` paramter.

In [17]:
gme.train(train_TSs, test_size = 0.1)

Information about the training process for each GM in the ensemble can be access using the `gm_info` attribute.  This attribute will contain a list of dictionaries; one dictionary for each GM in the ensemble.  Since we set `splits = 3` and `replicate = 1`, we should expect this list to have `splits * replicate = 3*1 = 3` dictionaries.

In [18]:
len(gme.gm_info)

3

Let's take a look at each of those dictionaries now:

In [19]:
for i in range(len(gme.gm_info)):
    pprint.pprint(gme.gm_info[i])
    print()

{'test_info': [      smape     sbias  exceed_0.05  exceed_0.95  exceed_0.99  step  idx  epoch
0  1.426815 -0.709916     0.193548     0.741935     0.838710     0   15      0
1  1.497390 -0.314829     0.258065     0.548387     0.677419     0    5      0],
 'train_loss_monitor': [0.095703565],
 'train_loss_val': [0.39477719366550446],
 'valid_fcst_monitor': [],
 'valid_loss_monitor': [{'epoch': 0}]}

{'test_info': [      smape     sbias  exceed_0.05  exceed_0.95  exceed_0.99  step  idx  epoch
0  1.356964 -0.911951     0.225806     0.677419     0.709677     0   15      0
1  1.381297 -0.269324     0.451613     0.548387     0.612903     0    5      0],
 'train_loss_monitor': [0.10445643],
 'train_loss_val': [0.23502697050571442],
 'valid_fcst_monitor': [],
 'valid_loss_monitor': [{'epoch': 0}]}

{'test_info': [      smape     sbias  exceed_0.05  exceed_0.95  exceed_0.99  step  idx  epoch
0  1.476115 -1.043133     0.193548     0.709677     0.709677     0   15      0
1  1.622907 -0.328690     

We can see that each dictionary has a key called 'test_info', which stores a list of DataFrames showing how each GM in the ensemble performs on each example of the test data.  Each DataFrame in the list represents a single epoch.

Since we set `epoch_num = 1` in the parameters, each list will only have one DataFrame.  Since the length of `train_TSs` is 20 and we set `test_size = 0.1`, there are two elements in our test set.  For each item in the test data set, in addition to the symmetric MAPE and bias metrics, the DataFrame shows the percentage of forecasted values that exceed all of the specified quantiles (in the `GMParam` object) other than the median.  Since the quantiles we are looking at other than the median are 0.05, 0.95 and 0.95, we get additional metrics `exceed_0.05`, `exceed_0.95`, and `exceed_0.99`.  

Let's take a look at one of these DataFrames.

In [20]:
gme.gm_info[0]['test_info'][0]

,smape,sbias,exceed_0.05,exceed_0.95,exceed_0.99,step,idx,epoch
0,1.426815,-0.709916,0.193548,0.741935,0.838710,0,15,0
1,1.497390,-0.314829,0.258065,0.548387,0.677419,0,5,0


After training the `GMEnsemble` object, you now can use it to generate forecasts. Similar to the `GMModel` object, the input of the `predict` function can either be a single `TimeSeriesData` or a list/dictionary of them and you must also specify the number of steps you wish to forecast.

In [21]:
fcsts=gme.predict(test_TSs, steps = 3)

As above with `GMModel`, this generates a dictionary with 5 keys, one for each time series in `test_TSs`.

In [22]:
fcsts.keys()

dict_keys([0, 1, 2, 3, 4])

Let's take a look at one of the forecasts.  

In [23]:
fcsts[2]

,fcst_quantile_0.5,fcst_quantile_0.05,fcst_quantile_0.95,fcst_quantile_0.99,time
0,0.952329,0.748380,1.068517,1.323682,2020-07-29
1,-0.375372,-0.740093,-0.506364,-0.027362,2020-07-30
2,-0.511230,-0.508175,-0.045864,-0.105266,2020-07-31


We can save a `GMEnsemble` to a file similarly to how we did for `GMModel`

In [24]:
# save model
gme.save_model("gme_example_1.p")

# load model
gme2 = load_gmensemble_from_file("gme_example_1.p")

# remove the saved model
os.remove("gme_example_1.p")

# generate forecasts
fcsts2=gme.predict(test_TSs, steps = 3)
fcsts2[2]

,fcst_quantile_0.5,fcst_quantile_0.05,fcst_quantile_0.95,fcst_quantile_0.99,time
0,0.952329,0.748380,1.068517,1.323682,2020-07-29
1,-0.375372,-0.740093,-0.506364,-0.027362,2020-07-30
2,-0.511230,-0.508175,-0.045864,-0.105266,2020-07-31


When encoding a `GMEnsemble` into a JSON string, we need to clear the `gm_info` attribute because it contains dictionaries with `pd.DataFrame` objects, which are not serializable.  Otherwise the code is the same as for `GMModel`.

In [25]:
# encode model into json string

gme.gm_info=None # Need to remove this because pd.DataFrame is not serilizable
gme_str = global_model_to_json(gme)

# load model from json string
gme3 = load_global_model_from_json(gme_str)

# generate forecasts
fcsts3=gme.predict(test_TSs, steps = 3)
fcsts3[2]

,fcst_quantile_0.5,fcst_quantile_0.05,fcst_quantile_0.95,fcst_quantile_0.99,time
0,0.952329,0.748380,1.068517,1.323682,2020-07-29
1,-0.375372,-0.740093,-0.506364,-0.027362,2020-07-30
2,-0.511230,-0.508175,-0.045864,-0.105266,2020-07-31


### 2.4 Backtesting with `GMBacktester`  

The `GMBacktester` object helps evaluate the hyperparameter setting (i.e., the `GMParam` object). Here is a list of some of the attributes:
* **data**: `list[TimeSeriesData` or `dict[TimeSeriesData]`, A list or a dictionary of time series objects for training and validation;
* **gmparam**: `GMParam`, the parameters that we're testing;
* **backtest_timestamp**: `list[str]` or `list[pd.Timestamp]`, timestamps used to split the time series for training and validation;
* **splits**: `int`, the number of sub-datasets into which the training data is to be partitioned.  Default is 3;
* **overlap**: `bool`,  Whether or not sub-datasets overlap with each other or not. If True, each training examples appears in `splits-1` sub-datasets.  Otherwise, each training example appears in only 1 sub-dataset.  Default is True;
* **replicate**: `int`, The number of GMs in the ensemble to be trained on each sub-dataset.  Default is 1;
* **test_size**: `float`, The proportion of the input `data` to be used for testing.  Default is 0.1.


For the full list of attributes, please see our documents.

In [26]:
from kats.models.globalmodel.backtester import GMBackTester

Now we can initialize the `GMBackTester` object as follows.

In [27]:
gbm = GMBackTester(train_TSs, gmparam, backtest_timestamp = ['2020-08-10'], test_size=0.1)

Now one can run backtesting using the `run_backtest` function.  This will give you a DataFrame that shows the evaluation metrics for each GM that was generated for each time series in our test data set.  For each backtesting date, number of GMs created is equal to `splits*replicate`, and the backtester also calculates the evaluation metrics for an ensemble that takes the median of each of these GMs.

In [28]:
backtest_df = gbm.run_backtest()
backtest_df

,smape,sbias,exceed_0.05,exceed_0.95,exceed_0.99,model_num,step,idx,type,backtest_ts
0,1.569330,-0.915494,0.258065,0.516129,0.451613,0.0,0,4.0,single,2020-08-10
1,1.540026,-0.601855,0.225806,0.419355,0.419355,1.0,0,4.0,single,2020-08-10
2,1.441183,-0.266776,0.290323,0.516129,0.483871,2.0,0,4.0,single,2020-08-10
3,1.472027,-0.670000,0.258065,0.516129,0.451613,NaN,0,NaN,ensemble,2020-08-10
4,1.617912,-0.771883,0.354839,0.612903,0.580645,0.0,0,19.0,single,2020-08-10
5,1.486416,-0.549728,0.290323,0.580645,0.548387,1.0,0,19.0,single,2020-08-10
6,1.496789,-0.212599,0.387097,0.645161,0.580645,2.0,0,19.0,single,2020-08-10
7,1.488076,-0.621214,0.354839,0.612903,0.580645,NaN,0,NaN,ensemble,2020-08-10
8,1.321206,-0.328787,0.838710,0.032258,0.032258,0.0,1,19.0,single,2020-08-10
9,1.483964,-0.309167,0.774194,0.032258,0.032258,1.0,1,19.0,single,2020-08-10


Let's take a look at the backtesting data for a single one of the GMs that we just trained.  We can do so as follows.

In [29]:
backtest_df[backtest_df.model_num == 1]

,smape,sbias,exceed_0.05,exceed_0.95,exceed_0.99,model_num,step,idx,type,backtest_ts
1,1.540026,-0.601855,0.225806,0.419355,0.419355,1.0,0,4.0,single,2020-08-10
5,1.486416,-0.549728,0.290323,0.580645,0.548387,1.0,0,19.0,single,2020-08-10
9,1.483964,-0.309167,0.774194,0.032258,0.032258,1.0,1,19.0,single,2020-08-10
13,1.318204,-0.651993,0.451613,0.129032,0.129032,1.0,2,19.0,single,2020-08-10
17,1.082353,-0.330191,0.032258,0.064516,0.064516,1.0,3,19.0,single,2020-08-10


Since the test time series have differnet lengths, we may need a different number of prediction steps for each time series.  The table we see above shows a different row for each training step.  In addition to the length of each time series, the number of training steps is determined by the `fcst_window` parameter in the `GMParam` object.  If we want to get a single metric for each time series, we can do the following aggregation.

In [30]:
backtest_df[backtest_df.model_num == 1].drop('step', axis=1).groupby('idx').mean()

,smape,sbias,exceed_0.05,exceed_0.95,exceed_0.99,model_num
idx,,,,,,
4.0,1.540026,-0.601855,0.225806,0.419355,0.419355,1.0
19.0,1.342734,-0.460270,0.387097,0.201613,0.193548,1.0


Let's take a look at the metrics calculated for only the GM Ensemble.

In [31]:
# TODO: refresh after diff fixing this lands
backtest_df[backtest_df.type == 'ensemble']

,smape,sbias,exceed_0.05,exceed_0.95,exceed_0.99,model_num,step,idx,type,backtest_ts
3,1.472027,-0.670000,0.258065,0.516129,0.451613,NaN,0,NaN,ensemble,2020-08-10
7,1.488076,-0.621214,0.354839,0.612903,0.580645,NaN,0,NaN,ensemble,2020-08-10
11,1.431047,-0.308835,0.870968,0.032258,0.032258,NaN,1,NaN,ensemble,2020-08-10
15,1.287114,-0.525685,0.451613,0.193548,0.258065,NaN,2,NaN,ensemble,2020-08-10
19,1.074946,-0.315819,0.032258,0.064516,0.096774,NaN,3,NaN,ensemble,2020-08-10


There are some constraints on how we pick the backtesting days.  Specifically, we need to have enough training days based on the the settings that we specify in the `GMParam` object.  We can see how many days that is using the `min_train_length` attribute.

In [32]:
gbm.min_train_length

90

Based on our current parameters, this tells us we need at least 90 data points for training.  If we try a date sooner than 90 days out from the first date, we will get an error.  The first date in each of our training time series is '2020-05-06', so the folowing does not work because '2020-08-01' is less than 90 days out.

In [33]:
# THIS DOES NOT WORK

# gbm = GMBackTester(train_TSs, gmparam, backtest_timestamp = ['2020-08-01'], test_size=0.1)

# gbm.run_backtest()

## 3. Using Pretrained Global Model or Global Ensemble


In Kats, we provide two pre-trained daily `GMEnsemble` objects (one is S2S-GME and and the other one is RNN-GME). Both of them are trained with M4 dataset. One can use them for forecasting exploration or benchmark.  

We can load these ensembles the same way we loaded the ensembles we created in section 2.3 using the `load_gmensemble_from_file` function.

In [34]:
# TODO: decide if we want to keep this section
gme_rnn = load_gmensemble_from_file("../kats/models/globalmodel/pretrained_daily_rnn.p")
gme_rnn

ERROR:root:Fail to load GMEnsemble from ../kats/models/globalmodel/pretrained_daily_rnn.p with Exception [Errno 2] No such file or directory: '../kats/models/globalmodel/pretrained_daily_rnn.p'.


ValueError: Fail to load GMEnsemble from ../kats/models/globalmodel/pretrained_daily_rnn.p with Exception [Errno 2] No such file or directory: '../kats/models/globalmodel/pretrained_daily_rnn.p'.

Once we have loaded the ensemble, we can use it to make predictions the same way we did with our own forecasts.

In [ ]:
fcsts = gme_rnn.predict(test_TSs, steps = 3)
fcsts[3]